# IMPORT

Basic prompts

In [121]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate , SystemMessagePromptTemplate , HumanMessagePromptTemplate 
from langchain.document_loaders import TextLoader

In [122]:
import os
from dotenv import load_dotenv

load_dotenv()
llm_model = ChatGoogleGenerativeAI(api_key=os.environ[("GOOGLE_API_KEY")],temperature=0.7,model="gemini-2.5-flash")

In [123]:
from langchain.chains import LLMChain
beginning_prompt = ChatPromptTemplate(messages=[SystemMessagePromptTemplate.from_template("You are a nice chatbot ,understand the user task title and date, rephrase the sentence in correct way like prompt to next llm"),
                                                HumanMessagePromptTemplate.from_template("{user_input}")])
layer_1_prompt = LLMChain(llm=llm_model , prompt=beginning_prompt , output_key = "rephrase")

middleware prompts

In [128]:
query_prompt = ChatPromptTemplate(messages=[SystemMessagePromptTemplate.from_template("you are a SQL query generator AI., you name is KnotAI,generate the query based on the table schema , tasks title & date .Table name is tasks it has user_email , id.return in JSON object formation key value in 'answer',Remove markdown ```json and extra text after JSON"),
                                            HumanMessagePromptTemplate.from_template("{rephrase}")]) 

layer_2_prompt = LLMChain(llm=llm_model , prompt=query_prompt , output_key = "final_output")

In [ ]:

from langchain_core.output_parsers import SimpleJsonOutputParser

parser = SimpleJsonOutputParser()
chains = layer_1_prompt | layer_2_prompt  

res= chains.invoke("Add a task jogging at 12/12/1212")
final_response = parser.parse(res["final_output"])

{'answer': "INSERT INTO tasks (user_email, task_name, task_date) VALUES ('your_user_email@example.com', 'jogging', '1212-12-12');"}


In [135]:
for i in chains.stream("Add a task jogging at 12/12/1212"):
    print(i, end="",flush=True)

{'user_input': 'Add a task jogging at 12/12/1212', 'rephrase': 'Certainly! Here\'s that task rephrased as a clear instruction for another LLM:\n\n**"Please add \'jogging\' to my schedule on December 12th, 1212."**', 'final_output': '{"answer": "INSERT INTO tasks (user_email, description, due_date) VALUES (\'user@example.com\', \'jogging\', \'1212-12-12\');"}'}